In [3]:
import params as par
import source
import geopandas as gpd
import matplotlib.pyplot as plt
plt.style.use('default')

In [4]:
#Verify Parameters in `params.py`
par.verify()

CENSUS_FILE = None does not exist
TOWN_FILE = None does not exist
EDGES_FILE = None does not exist
NODES_FILE = None does not exist


In [5]:
#Verify AOS
aos = gpd.read_file(par.AOS_FILE).to_crs(par.WK_CRS)
print(aos.crs)
aos.explore()

epsg:4326


In [20]:
import osmnx as ox

aos_gdf = aos
area = {
        "north": aos_gdf.total_bounds[3],
        "south": aos_gdf.total_bounds[1],
        "east": aos_gdf.total_bounds[2],
        "west": aos_gdf.total_bounds[0],
    }

G = source.preprocess.createGraph(area=area, crs=par.PJ_CRS, ntype=par.OSM_NTYPE, plot=False)
n, e = ox.graph_to_gdfs(G)

The graph was saved as 'graph.gpkg' with projection 6691.
This is a simplified walk type OSM network.
6392 edges in graph. The edges file was saved as ESRI shapefile
Database created!


In [19]:
e[e['osmid'].islist()]

osmid       highway  oneway reversed      length  \
u v   key                                                                      
0 1   0                 116068825      tertiary   False     True   58.878661   
  2   0                  76022882  unclassified   False     True  174.408964   
      1    [550057170, 550057171]         trunk   False     True   35.718130   
  917 0                 116068237  unclassified   False     True  775.885728   
  253 0    [106356196, 110920335]         trunk   False    False  355.356504   

                                                    geometry   u_original  \
u v   key                                                                   
0 1   0    LINESTRING (445633.591 4136183.470, 445647.253...    897114849   
  2   0    LINESTRING (445633.591 4136183.470, 445647.253...    897114849   
      1    LINESTRING (445633.591 4136183.470, 445647.253...    897114849   
  917 0    LINESTRING (445633.591 4136183.470, 445634.210...   1309825836   
  253 0    LINESTRING (445633.591 4136183.470, 445619.835...  10128911826   

           v_original lanes  ref          name bridge tunnel service  
u v   key                                                             
0 1   0    1309824615   NaN  NaN           NaN    NaN    NaN     NaN  
  2   0     897114901   NaN  NaN           NaN    NaN    NaN     NaN  
      1     897114901     2   49         国道49号    NaN    NaN     NaN  
  917 0    1309821950   NaN  NaN           NaN    NaN    NaN     NaN  
  253 0    1264976014     2   49  [金山橋, 国道49号]    yes    NaN     NaN

In [15]:
e.drop_duplicates(subset=['u_original','v_original'],inplace=True)


In [16]:
e.reset_index(inplace=True)
e.head()

,index,u,v,key,osmid,highway,oneway,reversed,length,geometry,u_original,v_original,lanes,ref,name,bridge,tunnel,service
0,0,0,1,0,116068825,tertiary,False,True,58.878661,"LINESTRING (445633.591 4136183.470, 445647.253...",897114849,1309824615,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,2,0,76022882,unclassified,False,True,174.408964,"LINESTRING (445633.591 4136183.470, 445647.253...",897114849,897114901,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0,917,0,116068237,unclassified,False,True,775.885728,"LINESTRING (445633.591 4136183.470, 445634.210...",1309825836,1309821950,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0,253,0,"[106356196, 110920335]",trunk,False,False,355.356504,"LINESTRING (445633.591 4136183.470, 445619.835...",10128911826,1264976014,2,49,"[金山橋, 国道49号]",yes,NaN,NaN
4,5,1,882,0,116068464,unclassified,False,False,59.463000,"LINESTRING (445631.268 4136136.639, 445683.369...",1309824615,1309828842,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
e.to_file('./edges.shp')

ValueError: Invalid field type <class 'list'>

In [ ]:
fig,axs = plt.subplots(1,7,sharex=True,sharey=True,figsize=(30,10))
plt.tight_layout()
shi_pol.plot(ax=axs[0])
# shi_town.plot(ax=axs[1],column='Val_GovCod')
# census.plot(ax=axs[2],column='M_TOTPOP_H',cmap='jet')
buildings.plot(ax=axs[3])
shelters.plot(ax=axs[4],column='HNJ_SITEI',cmap='jet')
mesh.plot(ax=axs[5],column='MESH4_ID',cmap='jet')
# edges.plot(ax=axs[6],color='grey',lw=1)
# nodes.plot(ax=axs[6],markersize=3,color='k')


In [ ]:
# aos = gpd.read_file(f'./case/{case}/aos.geojson',driver='GeoJSON')
aos = shi_pol

In [ ]:
c = census.clip(aos)
# sh = shelters.clip(aos)
m = mesh.clip(aos)
e = edges.clip(aos)
n = nodes.clip(aos)
# t = shi_town.clip(aos)

In [ ]:
fig,axs = plt.subplots(5,1,sharex=True,sharey=True,figsize=(10,20))
plt.tight_layout()
c.plot(ax=axs[0], column='M_TOTPOP_H',cmap='jet')
# sh.plot(ax=axs[1], column='capacity_per',cmap='jet')
m.plot(ax=axs[2], column='MESH4_ID',cmap='jet')
e.plot(ax=axs[3], color='grey',lw=1)
n.plot(ax=axs[3], markersize=3)
# t.plot(ax=axs[4], column='Val_GovCod')

In [ ]:
c.to_file(f'./case/{case}/aos_census.geojson',driver='GeoJSON')
# sh.to_file(f'./case/{case}/aos_shelters.geojson',driver='GeoJSON')
m.to_file(f'./case/{case}/aos_mesh.geojson',driver='GeoJSON')
e.to_file(f'./case/{case}/aos_edges_esri.geojson',driver='GeoJSON')
n.to_file(f'./case/{case}/aos_nodes_esri.geojson',driver='GeoJSON')
# t.to_file(f'./case/{case}/aos_town.geojson',driver='GeoJSON')

In [ ]:
import geopandas as gpd
gdf = gpd.read_file('./case/kochi/aos_census.geojson', driver="GeoJSON")
gdf.iloc[:,:-1]